Before we start doing anything, I think it's important to understand for NLP, this is the intuitive process on what we are trying to do when we are processing our data in the IMDB dataset:
1. Tokenization: break sentence into individual words
    - Before: `"PyTorch seems really easy to use!"`
    - After: `["PyTorch", "seems", "really", "easy", "to", "use", "!"]`
2. Building vocabulary: build an index of words associated with unique numbers
    - Before: `["PyTorch", "seems", "really", "easy", "to", "use", "!"]`
    - After: `{"Pytorch: 0, "seems": 1, "really": 2, ...}`
3. Convert to numerals: map words to unique numbers (indices)
    - Before: `{"Pytorch: 0, "seems": 1, "really": 2, ...}`
    - After: `[0, 1, 2, ...]`
4. Embedding look-up: map sentences (indices now) to fixed matrices
    - ```[[0.1, 0.4, 0.3],
       [0.8, 0.1, 0.5],
       ...]```

In [1]:
# Critical imports
import matplotlib.pyplot as plt
import torch
%matplotlib inline

In [2]:
# Set seed
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(1337)

In [3]:
# Set plotting style
plt.style.use(('dark_background', 'bmh'))
plt.rc('axes', facecolor='none')
plt.rc('figure', figsize=(16, 4))

In [4]:
from torchtext import data
import torch

# Create instances of fields
# The important field here is fix_length: all examples using this field will be padded to, or None for flexible sequence lengths
# We are fixing this because we will be using a FNN not an LSTM/RNN/GRU where we can go through uneven sequence lengths
max_len = 80
text = data.Field(sequential=True, fix_length=max_len, batch_first=True, lower=True, dtype=torch.long)
label = data.LabelField(sequential=False, dtype=torch.float)

In [5]:
# Calling splits() class method of datasets.IMDB to return a torchtext.data.Dataset object
from torchtext import datasets
ds_train, ds_test = datasets.IMDB.splits(text, label)

In [6]:
# Training and test set each 25k samples
# 2 fields due to the way we split above
print('train : ', len(ds_train))
print('test : ', len(ds_test))
print('train.fields :', ds_train.fields)

train :  25000
test :  25000
train.fields : {'label': <torchtext.data.field.LabelField object at 0x7f6494375978>, 'text': <torchtext.data.field.Field object at 0x7f6494375940>}


In [7]:
# Get validation set
import random
seed_num = 1337
ds_train, ds_valid = ds_train.split(random_state=random.seed(seed_num))

In [8]:
# Now we've training, validation and test set
print('train : ', len(ds_train))
print('valid : ', len(ds_valid))
print('valid : ', len(ds_test))

train :  17500
valid :  7500
valid :  25000


In [9]:
# Build vocabulary
# num_words = 25000
num_words = 25000
text.build_vocab(ds_train, max_size=num_words)
label.build_vocab(ds_train)

In [10]:
# Print vocab size
print('Vocabulary size: {}'.format(len(text.vocab)))
print('Label size: {}'.format(len(label.vocab)))

Vocabulary size: 25002
Label size: 2


In [11]:
# Print most common vocabulary text
most_common_samples = 10
print(text.vocab.freqs.most_common(most_common_samples))

[('the', 225938), ('a', 112571), ('and', 111513), ('of', 101389), ('to', 94175), ('is', 72550), ('in', 63886), ('i', 49428), ('this', 49096), ('that', 46809)]


In [12]:
# Print most common labels
print(label.vocab.freqs.most_common())

[('neg', 8835), ('pos', 8665)]


In [13]:
# Sample 0 label
ds_train[0].label

'neg'

In [14]:
# Sample 0 text: broken down into individual portions
ds_train[0].text

['first',
 'of',
 'all',
 'i',
 'just',
 'want',
 'to',
 'say',
 'that',
 'i',
 'love',
 'this',
 'show!!!',
 'but',
 'this',
 'episode...this',
 'episode',
 'makes',
 'a',
 'mockery',
 'of',
 'the',
 'entire',
 'show.<br',
 '/><br',
 '/>i',
 "don't",
 'know',
 'what',
 'they',
 'tried',
 'to',
 'achieve',
 'with',
 'this',
 'episode',
 'but',
 'they',
 'successfully',
 'created',
 'the',
 'worst',
 'episode',
 'in',
 'the',
 'entire',
 'series.<br',
 '/><br',
 '/>there',
 'is',
 'no',
 'story',
 'line,',
 'everything',
 'is',
 'chaotic',
 'and',
 'the',
 'jokes.....are',
 'crap.<br',
 '/><br',
 '/>the',
 'way',
 'they',
 'tried',
 'to',
 'answer',
 'some',
 'of',
 'the',
 'remaining',
 'questions',
 'in',
 'the',
 'game.....',
 'for',
 'example',
 '"how',
 'do',
 'the',
 'furlings',
 'look',
 'like"',
 'by',
 'creating',
 'that',
 'stupid',
 '"previously',
 'on..."......is',
 'simply',
 'embarrassing.<br',
 '/><br',
 '/>its',
 'clear',
 'that',
 'the',
 'writers',
 'are',
 'running',


In [15]:
# Sample 0 text: human readeable sample
def show_text(sample):
    print(' '.join(word for word in sample))
    
show_text(ds_train[0].text)

first of all i just want to say that i love this show!!! but this episode...this episode makes a mockery of the entire show.<br /><br />i don't know what they tried to achieve with this episode but they successfully created the worst episode in the entire series.<br /><br />there is no story line, everything is chaotic and the jokes.....are crap.<br /><br />the way they tried to answer some of the remaining questions in the game..... for example "how do the furlings look like" by creating that stupid "previously on..."......is simply embarrassing.<br /><br />its clear that the writers are running out of ideas and that is really too bad.


In [16]:
# Create and iterable object for our training, validation and testing datasets
# Batches examples of similar lengths together that minimizes amount of padding needed
batch_size = 64  # Change batch size from 1 to bigger number once explanation is done
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (ds_train, ds_valid, ds_test), batch_size=batch_size, sort_key=lambda x: len(x.text), repeat=False
)

In [17]:
# Check if iterator above is an iterable which should show True
import collections
isinstance(train_loader, collections.Iterable)

True

In [18]:
# What's inside this iteratable object? Our text and label although now everything is in machine format (not "words") but in numbers!
# The text we saw above becomes a matrix of size 1 x 80 represented by the fixed length we defined before that
list(train_loader)[0]


[torchtext.data.batch.Batch of size 64]
	[.label]:[torch.FloatTensor of size 64]
	[.text]:[torch.LongTensor of size 64x80]

In [19]:
# Alternative to above, this is much faster but the above code is easy to understand and implement
next(train_loader.__iter__())


[torchtext.data.batch.Batch of size 64]
	[.label]:[torch.FloatTensor of size 64]
	[.text]:[torch.LongTensor of size 64x80]

In [20]:
test_batch = next(train_loader.__iter__())

In [21]:
# What methods can we call on this batch object? Text and label
test_batch.fields

dict_keys(['label', 'text'])

In [22]:
# Let's break this down to check what's in a batch
test_batch.text

tensor([[     9,     61,    222,  ...,     37,      2,   2411],
        [ 17715,    173,      6,  ...,      1,      1,      1],
        [   944,   1341,    397,  ...,     44,   8097,   5461],
        ...,
        [     9,     54,    499,  ...,   6908,     11,    112],
        [     9,    222,      3,  ...,    161,     14,      8],
        [    49,      9,     82,  ...,      3,     56,   2888]])

In [23]:
# 1 comment per batch, each comment is limited to a size of 80 as we've defined
test_batch.text.size()

torch.Size([64, 80])

In [24]:
test_batch.label

tensor([ 1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,
         1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
         1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,
         0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,
         0.,  1.,  0.,  1.])

In [25]:
# Extremely weird problem in torchtext where BucketIterator returns a Batch object versus just a simple tuple of tensors containing our text index and labels
# So let's fix this with a new class FixBatchGenerator

class FixBatchGenerator:
    def __init__(self, dl, x_field, y_field):
        self.dl, self.x_field, self.y_field = dl, x_field, y_field
        
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        for batch in self.dl:
            X = getattr(batch, self.x_field)
            y = getattr(batch, self.y_field)
            yield (X,y)
            
train_loader, valid_loader, test_loader = FixBatchGenerator(train_loader, 'text', 'label'), FixBatchGenerator(valid_loader, 'text', 'label'), FixBatchGenerator(test_loader, 'text', 'label')

In [26]:
# Text index
print(next(train_loader.__iter__())[0])

# Text label
print(next(train_loader.__iter__())[1])

tensor([[   441,    103,    581,  ...,      1,      1,      1],
        [ 24336,    224,    399,  ...,    229,      3,   5546],
        [    49,     25,    208,  ...,    599,  11505,  17935],
        ...,
        [     9,     14,     23,  ...,   1069,     14,     50],
        [    10,      7,      3,  ...,      0,     38,    284],
        [   194,     71,      3,  ...,    186,      4,    312]])
tensor([ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,
         1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,
         1.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,
         0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,
         1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,
         1.,  0.,  1.,  0.])


In [27]:
import torch.nn as nn
import torch.nn.functional as F
   
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Embedding layer
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        # Linear function
        self.fc1 = nn.Linear(embedding_dim*embedding_dim, hidden_dim) 

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)
        embedded = embedded.view(-1, embedding_dim*embedding_dim)
        # Linear function  # LINEAR
        out = self.fc1(embedded)

        # Non-linearity  # NON-LINEAR
        out = F.relu(out)

        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        out = F.sigmoid(out)
    
        return out

In [28]:
print(max_len)
print(num_words)

80
25000


In [61]:
input_dim = num_words + 2
embedding_dim = max_len
hidden_dim = 128
output_dim = 1

# Instantiate model class and assign to object
model = FeedforwardNeuralNetModel(input_dim, embedding_dim, hidden_dim, output_dim)

# Push model to CUDA device if available
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function
criterion = nn.BCELoss()

# Optimizer
# learning_rate = 0.01
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [62]:
# Number of groups of parameters
print(len(list(model.parameters())))

5


In [63]:
# Print parameters
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([25002, 80])
torch.Size([128, 6400])
torch.Size([128])
torch.Size([1, 128])
torch.Size([1])


In [64]:
iter = 0
num_epochs = 100
for epoch in range(num_epochs):
#     print('-'*50)
#     print('Epoch {}'.format(epoch))
    for i, (samples, labels) in enumerate(train_loader):
        # Training mode
        model.train()
        
        # Load samples
        samples = samples.view(-1, max_len).to(device)
        labels = labels.view(-1, 1).to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(samples)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Testing mode
            model.eval()
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for samples, labels in valid_loader:
                # Load samples
                samples = samples.view(-1, max_len).to(device)
                labels = labels.view(-1).to(device)

                # Forward pass only to get logits/output
                outputs = model(samples)

                # This only works for one-hot encoded labels
                # _, predicted = torch.max(outputs.data, 1)
                
                # We use a threshold to define 
                predicted = outputs.ge(0.5).view(-1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted.type(torch.FloatTensor).cpu() == labels.type(torch.FloatTensor)).sum()
            
            accuracy = 100. * correct.item() / total
            # Print Loss
            print('Iter: {} Val Loss: {} val Accuracy: {}'.format(iter, loss.item(), round(accuracy, 2)))

Iter: 500 Val Loss: 0.5102349519729614 val Accuracy: 59.27
Iter: 1000 Val Loss: 0.0062254564836621284 val Accuracy: 60.49
Iter: 1500 Val Loss: 0.0006345253204926848 val Accuracy: 61.76
Iter: 2000 Val Loss: 0.0003830529167316854 val Accuracy: 61.91
Iter: 2500 Val Loss: 0.00028136116452515125 val Accuracy: 62.09
Iter: 3000 Val Loss: 0.00015922222519293427 val Accuracy: 62.27
Iter: 3500 Val Loss: 0.00014949310570955276 val Accuracy: 62.32
Iter: 4000 Val Loss: 7.680547423660755e-05 val Accuracy: 62.41
Iter: 4500 Val Loss: 5.3643689170712605e-05 val Accuracy: 62.51
Iter: 5000 Val Loss: 4.4746782805304974e-05 val Accuracy: 62.48
Iter: 5500 Val Loss: 2.787000084936153e-05 val Accuracy: 62.57
Iter: 6000 Val Loss: 2.1936943085165694e-05 val Accuracy: 62.56
Iter: 6500 Val Loss: 1.6642119589960203e-05 val Accuracy: 62.56
Iter: 7000 Val Loss: 1.2138345482526347e-05 val Accuracy: 62.6
Iter: 7500 Val Loss: 8.753620932111517e-06 val Accuracy: 62.68
Iter: 8000 Val Loss: 9.394418157171458e-06 val Accur

KeyboardInterrupt: 